# Question 1 (Association Rules)

<div class="alert alert-info"><font size=5>Question 1, part a: Number of Rules [5 points]</font>

If there are $m$ items (or features), there are $3^m-2^{m+1}+1$ different possible association rules.  Prove this.

You need a _clear_ explanation.  Note, associate rules must have a non-empty set on the left-hand and right-hand sides
</div>

## Your answer to part a here.
To be better understand why the number of possible rules is as given above, we can **add another set as "spare set"**, for those items in neither left-hand nor right-hand set.

For each item, it has to be in one of the sets (if it's not in the rule, it has to be in the spare set.), then there are 3 choices for each item. $m$ items will have $3^m$ different ways to put. Because left-hand and right-hand sets are non-empty, we need to throw those situations. When one of the sets is empty, each item will have 2 different ways to put, then there are $2^m$ ways. The number of possible ways will be $3^m-2*2^m$, but we throw the situation twice for when left and right hand sets are both empty, so we need to add $1$. Therefore, the number of possible ways is $3^m-2^{m+1}+1$.

<div class="alert alert-info"><font size=5>Question 1, part b: Association Rule Learning [15 points]</font>

In this question, you will write code to do association rule learning, as described in class.

The items will be represented by numbers (for ease and speed) with a separate
list of the names for each item.  `loaddata` (below) loads in a dataset and returns these three things: a list of the names of each item, a list of the examples, and the total number of items.  Each example is a set of numbers representing.  For example, for the toy problem in lecture, loaddata returns

`['Jurassic Park', 'Star Wars', 'Forrest Gump', 'Home Alone', 'Toy Story']`

`[[1, 2, 4], [1, 4], [1, 3, 4], [0, 1], [0, 3], [1, 3, 4], [0, 2, 3], [3], [1, 3, 4], [1]]`

`5`

You should use `set`s and `frozenset`s (core python data structures) in your code.  You can read more about them at https://docs.python.org/3/library/stdtypes.html#set

Write the functions `learnrules` and `writerules`, plus any additional helper functions you need.  Use the apriori algorithm to generate "large item lists" and the algorithm from class to find rules that meet the minimum support and confidence given.
</div>

In [1]:
from itertools import combinations #do not import anything else 
# (you may or may not use combinations -- up to you)

# prints out a set, nicely
# names is an optional list of the names for each of the (integer) items
def settostr(s,names=None):
    if names is None:
        elems = [str(e) for e in s]
    else:
        elems = [names[e] for e in s]
    return "{" + (", ".join(elems)) + "}"

In [2]:
# loads in data from filename, assuming the file format used for this assignment
def loaddata(filename):
    with open(filename) as f:
        nitems = int(f.readline())
        names = [f.readline().strip() for i in range(nitems)]
        nrows = int(f.readline())
        data = [[int(s) for s in f.readline().split()] for i in range(nrows)]
        f.close()
        return (names,data,nitems)        

In [3]:
def learnrules(numitems,data,minsupport,minconfidence):
    ### ADD YOUR CODE HERE (of course, feel free to add other functions!)
    ### Should return a list of rules.  
    ### Each rule should be a pair of two sets (lhs and rhs)
    
    # Generate candidate large itemsets
    def apriori_gen(Li_1, ii):
        # Generate C1
        if ii == 1:
            C1 = set()
            for d in data:
                for item in d:
                    item = frozenset([item])
                    C1.add(item)
            return C1
        # Generate Ci
        Ci = set()
        listL = list(Li_1)
        lenL = len(Li_1)
        for i in range(lenL):
            for j in range (1,lenL):
                I = list(listL[i])
                J = list(listL[j])
                I.sort()
                J.sort()
                if I[0:ii-2] == J[0:ii-2]:
                    item = listL[i] | listL[j]
                    # Throw away those sets that include non-frequent subsets
                    check = True
                    for k in item:
                        temp = item - frozenset([k])
                        if temp not in Li_1:
                            check = False
                            break
                    if check: Ci.add(item)
        return Ci
    
    # Generate Li from Ci
    def generate_Li(data,Ci,minsupport,s):
        Li = set()
        count = {}
        for d in data:
            for item in Ci:
                if item.issubset(d):
                    if item not in count:
                        count[item] = 1
                    else: count[item] += 1
        dnum = float(len(data))
        # Throw away those sets that have lower supports
        for item in count:
            if (count[item]/dnum) >= minsupport:
                Li.add(item)
                s[item] = count[item]/dnum
        return Li
    
    # Generate all large itemsets
    def apriori(numitems,data,minsupport):
        s = {}
        L1 = []
        L1 = generate_Li(data,apriori_gen(L1,1),minsupport,s)
        L = []
        L.append(L1)
        Li_1 = L1.copy()
        i = 1
        while len(Li_1) > 0:
            i += 1
            Ci = apriori_gen(Li_1,i)
            Li = generate_Li(data,Ci,minsupport,s)
            Li_1 = Li.copy()
            L.append(Li_1)
        return L,s
    
    # Learn the rules
    L,s = apriori(numitems,data,minsupport)
    rules = []
    subsetlist = []
    for i in range(len(L)):
        for frequent in L[i]:
            for subset in subsetlist:
                if subset.issubset(frequent) and subset != frequent:
                    temp = frequent-subset
                    confidence = s[frequent]/s[temp]
                    rule = (confidence,s[frequent],temp,subset)
                    if (confidence >= minconfidence) and (rule not in rules):
                        rules.append(rule)
            subsetlist.append(frequent)
    return rules

In [4]:
def writerules(rules,data,itemnames):
    ### ADD YOUR CODE HERE
    ## should print out each rule, *sorted by accuracy*, one per line
    ## each line should list the support, then the accuracy, then the rule
    ## to line up the columns nicely, use
    ##       "{:7.4f}".format(x)
    ## to print the floating point number in the variable x
    ## use settostr (above) to write out the itemsets
    rules.sort(reverse = True)
    for rule in rules:
        left = []
        right = []
        for item in rule[2]:
            left.append(itemnames[item])
        for item in rule[3]:
            right.append(itemnames[item])
        print("{:7.4f}".format(rule[1])," ","{:7.4f}".format(rule[0]),"  {",end = '')
        print(', '.join(left),end = '')
        print("} => {",end = '')
        print(', '.join(right),end = '')
        print("}")

In [5]:
# prints the rule set
def printruleset(datasetfilename,minsupport,minconfidence):
    (itemnames,data,numitems) = loaddata(datasetfilename)
    rules = learnrules(numitems,data,minsupport,minconfidence)
    writerules(rules,data,itemnames)

In [6]:
## toy dataset example
''' output should look like
 0.5000  1.0000    {Toy Story} => {Star Wars}
 0.3000  1.0000    {Star Wars, Home Alone} => {Toy Story}
 0.3000  1.0000    {Home Alone, Toy Story} => {Star Wars}
 0.5000  0.7143    {Star Wars} => {Toy Story}
 0.3000  0.6000    {Star Wars, Toy Story} => {Home Alone}
 0.3000  0.6000    {Toy Story} => {Home Alone}
 0.3000  0.6000    {Toy Story} => {Star Wars, Home Alone}
 0.3000  0.5000    {Home Alone} => {Toy Story}
 0.3000  0.5000    {Home Alone} => {Star Wars, Toy Story}
 0.3000  0.5000    {Home Alone} => {Star Wars}
'''
printruleset('toymovies.txt',0.3,0.5)

 0.5000    1.0000   {Toy Story} => {Star Wars}
 0.3000    1.0000   {Home Alone, Toy Story} => {Star Wars}
 0.3000    1.0000   {Star Wars, Home Alone} => {Toy Story}
 0.5000    0.7143   {Star Wars} => {Toy Story}
 0.3000    0.6000   {Star Wars, Toy Story} => {Home Alone}
 0.3000    0.6000   {Toy Story} => {Star Wars, Home Alone}
 0.3000    0.6000   {Toy Story} => {Home Alone}
 0.3000    0.5000   {Home Alone} => {Star Wars, Toy Story}
 0.3000    0.5000   {Home Alone} => {Toy Story}
 0.3000    0.5000   {Home Alone} => {Star Wars}


In [7]:
# the full groceries answer (should take a little under a minute to run)
printruleset('groceries.txt',0.01,0.5)

 0.0104    0.5862   {citrus fruit, root vegetables} => {other vegetables}
 0.0123    0.5845   {root vegetables, tropical fruit} => {other vegetables}
 0.0101    0.5824   {curd, yogurt} => {whole milk}
 0.0115    0.5736   {other vegetables, butter} => {whole milk}
 0.0120    0.5700   {root vegetables, tropical fruit} => {whole milk}
 0.0145    0.5630   {root vegetables, yogurt} => {whole milk}
 0.0123    0.5525   {domestic eggs, other vegetables} => {whole milk}
 0.0109    0.5245   {whipped/sour cream, yogurt} => {whole milk}
 0.0127    0.5230   {rolls/buns, root vegetables} => {whole milk}
 0.0135    0.5175   {pip fruit, other vegetables} => {whole milk}
 0.0151    0.5174   {tropical fruit, yogurt} => {whole milk}
 0.0223    0.5129   {other vegetables, yogurt} => {whole milk}
 0.0146    0.5070   {other vegetables, whipped/sour cream} => {whole milk}
 0.0122    0.5021   {rolls/buns, root vegetables} => {other vegetables}
 0.0129    0.5000   {root vegetables, yogurt} => {other vegetables